# Networkx ATLAS KG construction and RAG example
This notebook demonstrates the full streamlined process of creating a knowledge graph (KG) using the atlas-rag package and performing retrieval-augmented generation (RAG) with our created RAG methods.

## ATLAS KG Construction
It is suggested to use local hf model to run the KG construction code, as llm api service provider use optimized, lightweight models to reduce costs, which may sacrifice performance, and hence hard to have guaranteed performance. (for example from fp16 to fp8 etc.)

ATLAS KG construction consist of 5 steps:
- Triples Json Generation (Base KG Json)
- Convert Triples Json to Triples csv
- Conceptualize Entity in Triples csv
- Merge Concept CSV to Triples CSV
- Convert CSV to graphml for networkx to perform rag / to neo4j dumps for Billion KG RAG

In [1]:
from atlas_rag import TripleGenerator, KnowledgeGraphExtractor, ProcessingConfig
from openai import OpenAI
from transformers import pipeline
from configparser import ConfigParser
# Load OpenRouter API key from config file
config = ConfigParser()
config.read('config.ini')
model_name = "meta-llama/Llama-3.3-70B-Instruct"
# client = OpenAI(
#   base_url="https://api.deepinfra.com/v1/openai",
#   api_key=config['settings']['DEEPINFRA_API_KEY'],
# )

model_name = "meta-llama/Llama-3.1-8B-Instruct"
client = pipeline(
    "text-generation",
    model=model_name,
    device_map="auto",
)
keyword = 'Dulce'
output_directory = f'import/{keyword}'
triple_generator = TripleGenerator(client, model_name=model_name)

/home/httsangaj/miniconda3/envs/faiss-gpu/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.91s/it]


In [2]:
kg_extraction_config = ProcessingConfig(
      model_path=model_name,
      data_directory="tests",
      filename_pattern=keyword,
      batch_size=2,
      output_directory=f"{output_directory}",
)
kg_extractor = KnowledgeGraphExtractor(model=triple_generator, config=kg_extraction_config)

### Triples Generation (with OpenAI Package)

In [3]:
# construct entity&event graph
kg_extractor.run_extraction()

Found data files: ['Dulce.json']
Model: meta-llama/Llama-3.1-8B-Instruct


  0%|          | 0/1 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
100%|██████████| 1/1 [01:22<00:00, 82.38s/it]


In [4]:
# Convert Triples Json to CSV
kg_extractor.convert_json_to_csv()

Loading data from the json files
Number of files:  1


100%|██████████| 1/1 [00:00<00:00, 186.73it/s]

Processing file for file ids:  meta-llama_Llama-3.1-8B-Instruct_Dulce_output_20250605110208_1_in_1.json
Data to CSV completed successfully, start computing embeddings.


In [5]:
# Concept Generation
kg_extractor.generate_concept_csv_temp(batch_size=64)

all_batches 3


Shard_0:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

Number of unique conceptualized nodes: 226
Number of unique conceptualized events: 96
Number of unique conceptualized entities: 76
Number of unique conceptualized relations: 61


In [6]:
kg_extractor.create_concept_csv()

Loading concepts...


54it [00:00, 32379.19it/s]


Loading concepts done.
Relation to concepts: 13
Node to concepts: 41
Processing triple nodes...


41it [00:00, 13831.45it/s]


Processing concept nodes...


100%|██████████| 165/165 [00:00<00:00, 113266.80it/s]


Processing triple edges...


43it [00:00, 35000.01it/s]


# Choice 1: Convert to graphml for networkx rag

In [7]:
# convert csv to graphml for networkx
kg_extractor.convert_to_graphml()

## ATLAS Multihop QA

In [ ]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  # Set to the GPU you want to use, or '0' for the first GPU
import torch
num_gpus = torch.cuda.device_count()
print("number of GPUs available:", torch.cuda.device_count())
for i in range(num_gpus):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

number of GPUs available: 4
GPU 0: NVIDIA L20
GPU 1: NVIDIA L20
GPU 2: NVIDIA L20
GPU 3: NVIDIA L20


In order to perform RAG, one need to first create embeddings & faiss index for constructed KG

[There maybe performance difference in using AutoModel and Sentence Transformer for NV-Ebmed-v2]

In [ ]:
from sentence_transformers import SentenceTransformer
from atlas_rag.retriever import NvEmbed
from transformers import AutoModel
# Load the SentenceTransformer model
encoder_model_name = "nvidia/NV-Embed-v2"
# sentence_model = SentenceTransformer(encoder_model_name, trust_remote_code=True, model_kwargs={'device_map': "auto"})
# sentence_model.max_seq_length = 32768
# sentence_model.tokenizer.padding_side="right"
sentence_model = AutoModel.from_pretrained(encoder_model_name, trust_remote_code=True, device_map="auto")
sentence_encoder = NvEmbed(sentence_model)

/home/httsangaj/miniconda3/envs/faiss-gpu/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/httsangaj/projects/AutoSchemaKG/atlas_rag/billion/prompt_template.py:27: SyntaxWarning: invalid escape sequence '\('
  "content": "[[ ## question ## ]]\nSolve \(x^2 - 5x + 6 = 0\)."
/home/httsangaj/projects/AutoSchemaKG/atlas_rag/billion/prompt_template.py:167: SyntaxWarning: invalid escape sequence '\('
  "content": """[[ ## question ## ]]
Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.30s/it]


In [10]:
from openai import OpenAI
from atlas_rag.reader import LLMGenerator
from configparser import ConfigParser
# Load OpenRouter API key from config file
config = ConfigParser()
config.read('config.ini')
# reader_model_name = "meta-llama/llama-3.3-70b-instruct"
reader_model_name = "meta-llama/Llama-3.3-70B-Instruct"
client = OpenAI(
  # base_url="https://openrouter.ai/api/v1",
  # api_key=config['settings']['OPENROUTER_API_KEY'],
  base_url="https://api.deepinfra.com/v1/openai",
  api_key=config['settings']['DEEPINFRA_API_KEY'],
)
llm_generator = LLMGenerator(client=client, model_name=reader_model_name)

In [ ]:
from atlas_rag import create_embeddings_and_index
keyword = 'musique'
working_directory = f'/data/httsangaj/atomic-rag/8b'
data = create_embeddings_and_index(
    sentence_encoder=sentence_encoder,
    model_name = 'nvidia/NV-Embed-v2',
    working_directory=working_directory,
    keyword=keyword,
    include_concept=True,
    include_events=True,
    normalize_embeddings= True,
    text_batch_size=64,
    node_and_edge_batch_size=64,
)

In [ ]:
from atlas_rag.evaluation import BenchMarkConfig
benchmark_config = BenchMarkConfig(
    dataset_name= 'musique',
    question_file= "benchmark_data/musique.json",
    include_concept=True,
    include_events=True,
    reader_model_name=reader_model_name,
    encoder_model_name=encoder_model_name,
    number_of_samples=-1, # -1 for all samples
)

In [ ]:
from atlas_rag import setup_logger
logger = setup_logger(benchmark_config)

In [ ]:
# Initialize desired RAG method for benchmarking
from atlas_rag.retriever import HippoRAG2Retriever
hipporag2_retriever = HippoRAG2Retriever(
    llm_generator=llm_generator,
    sentence_encoder=sentence_encoder,
    data = data,
    logger=logger
)

In [ ]:
# start benchmarking
from atlas_rag.evaluation import RAGBenchmark
benchmark = RAGBenchmark(config=benchmark_config, logger=logger)
benchmark.run([hipporag2_retriever], llm_generator=llm_generator)

# Choice 2: Convert to neo4j dumps

In [3]:
from sentence_transformers import SentenceTransformer
from atlas_rag.retriever import SentenceEmbedding, NvEmbed
# use sentence embedding if you want to use sentence transformer
# use NvEmbed if you want to use NvEmbed-v2 model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
sentence_encoder = SentenceEmbedding(sentence_model)

/home/httsangaj/miniconda3/envs/faiss-gpu/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# add numeric id to the csv so that we can use vector indices
kg_extractor.add_numeric_id()

# compute embedding
kg_extractor.compute_embedding(sentence_encoder) # default encoder_model_name="all-MiniLM-L12-v2" other options: "nvidia/NV-Embed-v2"
# kg_extractor.compute_embedding(encoder_model_name="all-MiniLM-L12-v2")
# kg_extractor.compute_embedding(encoder_model_name="nvidia/NV-Embed-v2")

# create faiss index
kg_extractor.create_faiss_index() # default index_type="HNSW,Flat", other options: "IVF65536_HNSW32,Flat" for large KG
# kg_extractor.create_faiss_index(index_type="HNSW,Flat")
# kg_extractor.create_faiss_index(index_type="IVF65536_HNSW32,Flat")


['name:ID', 'type', 'concepts', 'synsets', ':LABEL']


Adding numeric ID: 41it [00:00, 76805.03it/s]


[':START_ID', ':END_ID', 'relation', 'concepts', 'synsets', ':TYPE']


Adding numeric ID: 43it [00:00, 46483.27it/s]

['text_id:ID', 'original_text', ':LABEL']


Adding numeric ID: 1it [00:00, 2739.58it/s]


Processed chunk 0 (100000 rows)
Total number of embeddings: 41
Conversion complete!
Processed chunk 0 (100000 rows)
Total number of embeddings: 1
Conversion complete!
Processed chunk 0 (100000 rows)
Total number of embeddings: 43
Conversion complete!
Dimension: 384
Dimension: 384
Dimension: 384


## Install Neo4j Server

Go to the AutoschemaKG/neo4j_scripts directory

```sh get_neo4j_demo.sh```

Then there a neo4j server is install in the directory: neo4j-server-dulce

Start the newly instealled empty Neo4j server for testing

```sh start_neo4j_demo.sh```



## Config Neo4j Server

Stop the server first before config and import data

```sh stop_neo4j_demo.sh```


Copy the ```AutoschemaKG/neo4j_scripts/neo4j.conf``` file to the conf directory of the Neo4j server (```neo4j-server-dulce/conf```). Then, update the following settings as needed: 1.Set dbms.default_database to the desired dataset name, such as ```wiki-csv-json-text```, ```pes2o-csv-json-text```, or ```cc-csv-json-text```. In this case we make it ```dulce-csv-json-text``` 2.Configure the Bolt, HTTP, and HTTPS connectors according to your requirements.

I have set up the config port to some random ports to avoid port conflicts in ```neo4j-server-dulce/conf/neo4j.conf``` .

 
``` 
# Bolt connector
server.bolt.enabled=true
#server.bolt.tls_level=DISABLED
server.bolt.listen_address=0.0.0.0:8612
server.bolt.advertised_address=:8612

# HTTP Connector. There can be zero or one HTTP connectors.
server.http.enabled=true
server.http.listen_address=0.0.0.0:7612
server.http.advertised_address=:7612

# HTTPS Connector. There can be zero or one HTTPS connectors.
server.https.enabled=false
server.https.listen_address=0.0.0.0:7781
server.https.advertised_address=:7781
```


## Import Data
We use the admin import method to import data, which is the fastest way. Other methods are too slow for large graphs.


## Load the CSV files into Neo4j

We try to import data from previously constructed csv files with numeric ids. All the csv files are in ```import/Dulce```. 
In total six csv files for the nodes and edges of triples, text chunks, and concepts. 

``` shell
./neo4j-server-dulce/bin/neo4j-admin database import full dulce-csv-json-text \
    --nodes ./import/Dulce/triples_csv/triple_nodes_Dulce_from_json_without_emb_with_numeric_id.csv \
    --nodes ./import/Dulce/triples_csv/text_nodes_Dulce_from_json_with_numeric_id.csv \
    --nodes ./import/Dulce/concept_csv/concept_nodes_Dulce_from_json_with_concept.csv \
    --relationships ./import/Dulce/triples_csv/triple_edges_Dulce_from_json_without_emb_with_numeric_id.csv \
    --relationships ./import/Dulce/triples_csv/text_edges_Dulce_from_json.csv \
    --relationships ./import/Dulce/concept_csv/concept_edges_Dulce_from_json_with_concept.csv  \
    --overwrite-destination \
    --multiline-fields=true \
    --id-type=string \
    --verbose --skip-bad-relationships=true
```

When this is finished, you can see the following notifications

```shell
IMPORT DONE in 2s 475ms. 
Imported:
  1183 nodes
  2519 relationships
  6743 properties
Peak memory usage: 1.032GiB
```

Then you can start host it by running in ```./neo4j_scripts```

```sh start_neo4j_demo.sh```

When you see the following line, then it is working well.


```Started neo4j (pid:742490). It is available at http://0.0.0.0:7612```



If you want to use the python driver to run neo4j, you need to use port 8612. You can access http://0.0.0.0:7612 in browser as well to use the neo4j GUI. 

The default user is ```neo4j``` with password ```admin2024```. 


## ATLAS Billion Level RAG
The LargeKGRetriever is designed to perform retrieval on a billion-level graph. 

There is a trade-off between retrieval performance and speed; this serves as a proof of concept for a billion-level knowledge graph.

After successfully hosting the Neo4j database, you can run the provided Python script to host the RAG API:
```shell
python atlas_api_demo.py 
```

During the first startup of the API, it will create the necessary indexes and projection graphs in the Neo4j database for faster queries and computations. The time required for this process may vary depending on the size of the database. You can monitor the creation of these items in http://localhost:7612 by using the following commands:

To view the projected graphs:
```cypher
CALL gds.graph.list()
```
To view the indexes:
```cypher
SHOW INDEXES
```

The projected graph will be deleted after the database is shut down, while the indexes will not be removed.

After you saw: \
Index NodeNumericIDIndex created in 0.09 seconds \
Index TextNumericIDIndex created in 0.11 seconds \
Index EntityEventEdgeNumericIDIndex created in 0.02 seconds \
Projection graph largekgrag_graph created in 5.42 seconds 

You can perform rag as follows:

In [28]:
from openai import OpenAI

base_url ="http://0.0.0.0:10090/v1/"
client = OpenAI(api_key="EMPTY", base_url=base_url)

# knowledge graph en_simple_wiki_v0
message = [
    {
        "role": "system",
        "content": "You are a helpful assistant that answers questions based on the knowledge graph.",
    },
    {
        "role": "user",
        "content": "Question: Who is Alex Mercer?",
    }
]
response = client.chat.completions.create(
    model="llama",
    messages=message,
    max_tokens=2048,
    temperature=0.5,
    extra_body = {
        "retriever_config":{ # configure based on the size of your knowledge graph
            "topN": 5,
            "number_of_source_nodes_per_ner": 1,
            "sampling_area": 10 
        }
    }
)
print(response.choices[0].message.content)

According to the reference document (Passage 2), Alex Mercer is an agent of the Paranormal Military Squad and a key member of the team involved in Operation: Dulce. He is described as being "unfailingly determined on paper" but seems overwhelmed by the enormity of the mission. Despite his compliance with protocol, he has a natural inclination to question and explore beyond the given parameters, which creates an inner conflict within him. He is part of a team that includes other agents such as Taylor Cruz, Jordan Hayes, and Sam Rivera, each with their own perspectives and approaches to their mission.
